In [1]:
import numpy as np
import cv2

In [3]:
_, segmtrue = cv2.imreadmulti("data/training_fullmovie_segmentation_cellpose_v2_FORM.tif", [], cv2.IMREAD_ANYDEPTH)

In [4]:
def reindex_cell_labels(img):
    idx = np.unique(img)
    img_c = img.copy()
    for i, x in enumerate(idx):
        img_c[img_c == x] = i
    return (img_c.astype('uint16'))


# Recover ground truth into a single array
true_segs_test = np.zeros((125,512,512))
for f in range(125):
    true_segs_test[f,:,:] = reindex_cell_labels(segmtrue[f])

(125, 512, 512)


Tracing function

In [3]:
def tracking(labels):
    # input: labeling of the groundtruth of the cell segmentation 
    # output: tracking that needs to be corrected with napari in order to obtain the groundtruth of the tracing
    track = labels.copy()
    N_time = labels.shape[0]
    image1 = track[0,:,:]
    indices = np.unique(image1)

    for t in range(N_time -1):
        image1 = track[t,:,:].copy()
        image2 = track[t+1,:,:].copy()
        indices2 = np.unique(image2)
        
        for i, ind2 in enumerate(indices2) :

            if (ind2 != 0):

                # cach_cell2 represents the position of cell2 at time t+1
                cach_cell2 = image2.copy()
                cach_cell2[cach_cell2!=ind2] = 0
                cach_cell2[cach_cell2 == ind2] = 1

                # find what is at time t at the position of cell2 at time t+1    
                superposition = cach_cell2 * image1
                indicessup = np.unique(superposition)

                if (indicessup.shape[0] > 1):
                    superposition = superposition.reshape(1, -1)
                    # find the lineage that superposes the most with cell2
                    indsup = np.bincount(superposition[0][superposition[0]!=0]).argmax()
                    track[t+1][(cach_cell2 *track[t+1]) == ind2] = indsup
                else:
                    indices = np.append(indices, ind2)
    return track

In [23]:
track = tracking(true_segs_test)

<Labels layer 'test' at 0x1ec5af67d60>